In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic

In [20]:
# chat = ChatOpenAI()

# chat.predict("How many planets are there?")

In [8]:
chat = ChatOpenAI(temperature=0.1)

In [22]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [23]:
messages = [
    SystemMessage(content = "You are a geography expert. And you only reply in Spanish",),
    AIMessage(content= "Hola, mi nombre es Miguel"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name")

]

chat.predict_messages(messages)

AIMessage(content='Hola, mi nombre es Miguel. La distancia entre México y Tailandia es de aproximadamente 16,000 kilómetros.')

## 3.2 Prompt Templates

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [27]:
template = PromptTemplate.from_template("what is the distance between {country_a} and  {country_b},")

prompt = template.format(country_a="Korea", country_b="Japan")

chat.predict(prompt)

'The distance between Korea and Japan is approximately 900 kilometers (560 miles) across the Korea Strait.'

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "こんにちは、私の名前は{name}です"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        )
    ]
)

prompt = template.format_messages(
    language ="Japanese",
    name ="セツ",
    country_a = "Korea",
    country_b = "Japan"
)

chat.predict_messages(prompt)

AIMessage(content='韓国と日本の間の距離は約約200kmです。私の名前はセツです。どうぞよろしくお願いします。')

## 3.3 OutputParser and LCEL 

### OutputParser

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items =text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")
    

['Hello', 'how', 'are', 'you']

In [10]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked\
        will be answered with a comma seperated list of max {max_items} in lowercase.\
        Do NOT reply with anything else."),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items =10,
    question = "What are the plants?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['rose',
 'sunflower',
 'fern',
 'cactus',
 'tulip',
 'daisy',
 'orchid',
 'lavender',
 'succulent',
 'daffodil']

### LCEL

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items =text.strip().split(",")
        return list(map(str.strip, items))
    
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked\
        will be answered with a comma seperated list of max {max_items} in lowercase.\
        Do NOT reply with anything else."),
        ("human", "{question}"),
    ]
)

In [15]:
chain = template | chat | CommaOutputParser()
chain.invoke(
   {"max_items": 10,
    "question" : "What are the pokemons?"}
)

['bulbasaur',
 'ivysaur',
 'venusaur',
 'charmander',
 'charmeleon',
 'charizard',
 'squirtle',
 'wartortle',
 'blastoise',
 'pikachu']

## 3.4 Chaining Chains

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                  streaming=True, # showing what is going on during the process
                  callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. \
         You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {cuisine} food."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. \
         You find alternative ingredients and explain their preparation. \
         You don't radically modify the recipe. \
         If there is no alternative for a food just say you don't know how to recipe it."), 
        ("human", "{recipe}")
    ]
)


In [24]:
chef_chain = chef_prompt | chat
veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke(
    {
        "cuisine": "Korean",
    }
)

Great choice! How about making some delicious Bibimbap? Here's a simple recipe for you to try at home:

Ingredients:
- 2 cups cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound beef (you can use ribeye or sirloin), thinly sliced
- 4 eggs
- 4 tablespoons soy sauce
- 2 tablespoons sesame oil
- 2 tablespoons gochujang (Korean red chili paste)
- 2 cloves garlic, minced
- Salt and pepper to taste
- Sesame seeds for garnish
- Vegetable oil for cooking

Instructions:
1. Marinate the beef: In a bowl, mix the beef with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, minced garlic, salt, and pepper. Let it marinate for at least 30 minutes.
2. Prepare the vegetables: Blanch the spinach in boiling water for 1 minute, then rinse with cold water and squeeze out excess water. Season with a pinch of salt and a drizzle of sesame oil. Blanch the bean sprouts in boiling water for 2 minutes, then rinse wit

AIMessageChunk(content='To make this Bibimbap recipe vegetarian, you can make a few simple swaps:\n\n- Instead of beef, you can use tofu as a protein alternative. Extra-firm tofu works well for this dish. Cut the tofu into thin slices or cubes and marinate it in a mixture of soy sauce, sesame oil, minced garlic, salt, and pepper for at least 30 minutes before cooking.\n\n- For the eggs, you can omit them or use a plant-based egg alternative if desired.\n\n- To replace the meaty flavor of shiitake mushrooms, you can use a mix of other mushrooms like cremini or portobello mushrooms. Slice them and sauté them until tender.\n\n- If you prefer a vegan version, you can skip the eggs and adjust the Bibimbap sauce by using a vegan-friendly gochujang paste and ensuring the soy sauce you use is also vegan.\n\nBy making these ingredient swaps, you can enjoy a delicious vegetarian Bibimbap that stays true to the traditional flavors of the dish. Enjoy your homemade vegetarian Bibimbap!')